# 特徴量エンジニアリング テンプレート

このノートブックは、Kaggle競技における特徴量エンジニアリングのテンプレートです。

## 1. ライブラリのインポート

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

## 2. データの読み込み

In [ ]:
# データの読み込み
train = pd.read_csv("../data/raw/train.csv")
test = pd.read_csv("../data/raw/test.csv")

print(f"訓練データ形状: {train.shape}")
print(f"テストデータ形状: {test.shape}")

## 3. 欠損値の処理

In [ ]:
# 欠損値の処理
# 数値変数: 平均値で補完
numeric_cols = train.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    if train[col].isnull().sum() > 0:
        mean_val = train[col].mean()
        train[col].fillna(mean_val, inplace=True)
        test[col].fillna(mean_val, inplace=True)

# カテゴリ変数: 最頻値で補完
categorical_cols = train.select_dtypes(include=["object"]).columns
for col in categorical_cols:
    if train[col].isnull().sum() > 0:
        mode_val = train[col].mode()[0]
        train[col].fillna(mode_val, inplace=True)
        test[col].fillna(mode_val, inplace=True)

## 4. カテゴリ変数のエンコーディング

In [ ]:
# ラベルエンコーディング
from sklearn.preprocessing import LabelEncoder

le_dict = {}
for col in categorical_cols:
    le = LabelEncoder()
    # 訓練データとテストデータを結合してエンコーディング
    combined = pd.concat([train[col], test[col]], axis=0)
    le.fit(combined)
    train[col + "_encoded"] = le.transform(train[col])
    test[col + "_encoded"] = le.transform(test[col])
    le_dict[col] = le
    
print("ラベルエンコーディング完了")

## 5. 数値変数の変換

In [ ]:
# 対数変換（正の値のみ）
for col in numeric_cols:
    if (train[col] > 0).all():
        train[col + "_log"] = np.log1p(train[col])
        test[col + "_log"] = np.log1p(test[col])

# 標準化
scaler = StandardScaler()
scaled_cols = [col for col in numeric_cols if col != "target"]
if scaled_cols:
    train_scaled = scaler.fit_transform(train[scaled_cols])
    test_scaled = scaler.transform(test[scaled_cols])
    
    for i, col in enumerate(scaled_cols):
        train[col + "_scaled"] = train_scaled[:, i]
        test[col + "_scaled"] = test_scaled[:, i]

## 6. 新しい特徴量の作成

In [ ]:
# 特徴量の組み合わせ
# 例: 数値特徴量同士の四則演算
if len(numeric_cols) >= 2:
    col1, col2 = numeric_cols[0], numeric_cols[1]
    train[f"{col1}_plus_{col2}"] = train[col1] + train[col2]
    train[f"{col1}_minus_{col2}"] = train[col1] - train[col2]
    train[f"{col1}_mult_{col2}"] = train[col1] * train[col2]
    train[f"{col1}_div_{col2}"] = train[col1] / (train[col2] + 1e-8)
    
    test[f"{col1}_plus_{col2}"] = test[col1] + test[col2]
    test[f"{col1}_minus_{col2}"] = test[col1] - test[col2]
    test[f"{col1}_mult_{col2}"] = test[col1] * test[col2]
    test[f"{col1}_div_{col2}"] = test[col1] / (test[col2] + 1e-8)

## 7. 処理済みデータの保存

In [ ]:
# 処理済みデータの保存
os.makedirs("../data/processed", exist_ok=True)

# ターゲット変数の分離
if "target" in train.columns:
    X_train = train.drop("target", axis=1)
    y_train = train["target"]
else:
    X_train = train
    y_train = None

X_test = test

# 保存
X_train.to_csv("../data/processed/X_train_processed.csv", index=False)
X_test.to_csv("../data/processed/X_test_processed.csv", index=False)
if y_train is not None:
    y_train.to_csv("../data/processed/y_train.csv", index=False)

print("処理済みデータを保存しました")
print(f"X_train形状: {X_train.shape}")
print(f"X_test形状: {X_test.shape}")
if y_train is not None:
    print(f"y_train形状: {y_train.shape}")